In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
data=pd.read_csv("/content/gdrive/MyDrive/Classroom/googleplaystore_user_reviews.csv")

In [ ]:
data.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [ ]:
df=pd.concat([data.Translated_Review,data.Sentiment],axis=1)
df.dropna(axis=0,inplace=True)
df.head(10)

,Translated_Review,Sentiment
0,I like eat delicious food. That's I'm cooking ...,Positive
1,This help eating healthy exercise regular basis,Positive
3,Works great especially going grocery store,Positive
4,Best idea us,Positive
5,Best way,Positive
6,Amazing,Positive
8,"Looking forward app,",Neutral
9,It helpful site ! It help foods get !,Neutral
10,good you.,Positive
11,Useful information The amount spelling errors ...,Positive


In [ ]:
df.Sentiment=[0 if i=="Positive" else 1 if i== "Negative" else 2 for i in df.Sentiment]

In [ ]:
## Get the Independent Features

X=df.drop('Sentiment',axis=1)

In [ ]:
## Get the Dependent features
y=df['Sentiment']
y

In [ ]:
X.shape

(37427, 1)

In [ ]:
y.shape

(37427,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages['Translated_Review'][1]

'This help eating healthy exercise regular basis'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
    review = re.sub('[^a-zA-Z]', ' ', messages['Translated_Review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

### Embedding Representation

In [ ]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4976 1727 2027]
 [   0    0    0 ... 1564 1781 3295]
 [   0    0    0 ...  516 1854 2712]
 ...
 [   0    0    0 ... 3322  886 1075]
 [   0    0    0 ...  222 4571  385]
 [   0    0    0 ... 2844 1380 4647]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 3644,  631, 2946,  955, 1516,  955, 1428, 4976,
        955, 2658, 1275,  826, 4976, 1727, 2027], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=200
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(3,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 40, 200)           1000000   
_________________________________________________________________
lstm_22 (LSTM)               (None, 40, 100)           120400    
_________________________________________________________________
lstm_23 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 303       
Total params: 1,201,103
Trainable params: 1,201,103
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(37427, (37427,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((37427, 40), (37427,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=32)

Epoch 1/100
784/784 [==============================] - 15s 19ms/step - loss: 0.0143 - accuracy: 0.9948 - val_loss: 0.9403 - val_accuracy: 0.8689
Epoch 2/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0112 - accuracy: 0.9954 - val_loss: 1.0291 - val_accuracy: 0.8637
Epoch 3/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0142 - accuracy: 0.9950 - val_loss: 0.9710 - val_accuracy: 0.8684
Epoch 4/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0183 - accuracy: 0.9932 - val_loss: 0.8766 - val_accuracy: 0.8626
Epoch 5/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0148 - accuracy: 0.9945 - val_loss: 0.9427 - val_accuracy: 0.8653
Epoch 6/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0124 - accuracy: 0.9954 - val_loss: 0.9779 - val_accuracy: 0.8632
Epoch 7/100
784/784 [==============================] - 14s 18ms/step - loss: 0.0109 - accuracy: 0.9957 - val_loss: 1.0054 - val_ac

### Performance Metrics And Accuracy

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[7244,  421,  192],
       [ 452, 2131,  151],
       [ 306,  131, 1323]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8661646830216176